<a href="https://colab.research.google.com/github/junsookim1/Everday_Practice/blob/master/Koyang_Bike_Data(%EC%9B%94_%EB%8B%A8%EC%9C%84_%EA%B5%AC%EB%B6%84_(%EB%8C%80%EC%97%AC%2C%EB%B0%98%EB%82%A9)%EC%8A%A4%ED%85%8C%EC%9D%B4%EC%85%98_%EC%9D%B4%EB%8F%99_%ED%9A%9F%EC%88%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import missingno as msno
from haversine import haversine

In [ ]:
df_1 = pd.read_csv('월별 스테이션 이용 현황.csv')# 불러오기

In [ ]:
df_1

,Unnamed: 0,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호,추정 이동 거리,이용 시간(s),회원 번호,비회원 번호
0,0,2017-01,213,2017-01,260,0.0,791.0,164203,0.0
1,1,2017-01,231,2017-01,231,31039.0,2961.0,187551,0.0
2,2,2017-01,119,2017-01,0,NaN,3600.0,168994,0.0
3,3,2017-01,121,2017-01,133,15490.0,829.0,183971,0.0
4,4,2017-01,320,2017-01,259,0.0,912.0,167475,0.0
...,...,...,...,...,...,...,...,...,...
3142683,3142683,2019-12,222,2019-12,219,0.0,377.0,236957,0.0
3142684,3142684,2019-12,343,2019-12,311,0.0,240.0,0,4124665.0
3142685,3142685,2019-12,333,2020-01,322,0.0,657.0,0,4124667.0
3142686,3142686,2019-12,264,2020-01,210,0.0,685.0,180671,0.0


In [ ]:
# df.to_csv('스테이션 이용 현환(진행중).csv') # 저장
df = pd.read_csv('스테이션 이용 현환(진행중).csv')# 불러오기

In [ ]:
df # 이용회수를 알기 위해 필요한 데이터만 가져온다.

,Unnamed: 0,대여 시간(M),대여 스테이션 번호,반납 시간(M),반납 스테이션 번호
0,0,2017-01,213,2017-01,260
1,1,2017-01,231,2017-01,231
2,2,2017-01,119,2017-01,0
3,3,2017-01,121,2017-01,133
4,4,2017-01,320,2017-01,259
...,...,...,...,...,...
3142683,3142683,2019-12,222,2019-12,219
3142684,3142684,2019-12,343,2019-12,311
3142685,3142685,2019-12,333,2020-01,322
3142686,3142686,2019-12,264,2020-01,210


In [ ]:
df['대여 스테이션 번호']=df_1[['대여 스테이션 번호']].astype('str')

In [ ]:
df['대여 스테이션 번호'].apply(lambda x: x.count('213')).sum() # 총 이용회수를 구할 수 있다.

29771

In [ ]:
((df['대여 스테이션 번호']=='110')&(df['대여 시간(M)']=='2017-01')).sum() # 월별 이용 회수 구하기

numpy.int64

In [ ]:
df_station_num = df['대여 스테이션 번호'].unique().tolist()
df_station_num.sort() 
len(df_station_num)
dfdf_station_num = pd.DataFrame(df_station_num, columns=['이용 횟수(전체)'])
dfdf_station_num

,이용 횟수(전체)
0,101
1,103
2,104
3,105
4,106
...,...
151,349
152,350
153,351
154,992


In [ ]:
df_month_hour = df['대여 시간(M)'].unique().tolist()
df_month_hour

['2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12']

In [ ]:
Bike_Station_month_usage_status = [] # 월별 대여 스테이션 이용 현황
for i in df_month_hour:
    for j in df_station_num:
        Bike_Station_month_usage_status.append(((df['대여 시간(M)']==i)&(df['대여 스테이션 번호']==j)).sum())
# 2017년도 2018년도 이런식으로 이용 현황으로 dataframe 만들어 줄 것

In [ ]:
len(Bike_Station_month_usage_status) 

5616

In [ ]:
df['대여 스테이션 번호'].apply(lambda x: x.count('101')).sum()

numpy.int64

In [ ]:
# 총 대여 스테이션 이용 현황
total_Station_status = []
for i in df_station_num:
    total_Station_status.append(df['대여 스테이션 번호'].apply(lambda x: x.count(i)).sum())

KeyboardInterrupt: 

In [ ]:
len(total_Station_status)

156

In [ ]:
df_all = pd.read_csv('월별 스테이션 이용 현황(빈 리스트).csv')# 불러오기

In [ ]:
df_all

,대여 시간(M),대여 스테이션 번호
0,2017-01,101
1,2017-01,103
2,2017-01,104
3,2017-01,105
4,2017-01,106
...,...,...
5611,2019-12,349
5612,2019-12,350
5613,2019-12,351
5614,2019-12,992


In [ ]:
Bike_Station_month_usage_status

[80,
 33,
 152,
 27,
 62,
 112,
 69,
 97,
 144,
 134,
 87,
 133,
 192,
 151,
 148,
 43,
 114,
 92,
 41,
 151,
 95,
 90,
 95,
 156,
 29,
 79,
 131,
 28,
 66,
 56,
 74,
 95,
 138,
 50,
 85,
 119,
 119,
 81,
 43,
 53,
 70,
 68,
 32,
 76,
 29,
 41,
 45,
 20,
 0,
 0,
 0,
 225,
 158,
 540,
 176,
 158,
 185,
 376,
 756,
 751,
 466,
 363,
 650,
 893,
 34,
 605,
 410,
 230,
 242,
 372,
 428,
 77,
 142,
 421,
 511,
 446,
 380,
 698,
 410,
 487,
 416,
 261,
 446,
 355,
 188,
 127,
 167,
 47,
 95,
 492,
 466,
 254,
 133,
 291,
 193,
 162,
 386,
 697,
 615,
 81,
 299,
 151,
 143,
 569,
 410,
 0,
 0,
 0,
 0,
 0,
 175,
 346,
 290,
 83,
 514,
 504,
 483,
 387,
 1104,
 733,
 578,
 402,
 409,
 314,
 297,
 456,
 590,
 364,
 215,
 275,
 638,
 178,
 365,
 114,
 364,
 156,
 203,
 59,
 284,
 373,
 95,
 190,
 169,
 184,
 84,
 597,
 515,
 170,
 62,
 49,
 40,
 96,
 66,
 0,
 8,
 0,
 74,
 59,
 180,
 19,
 67,
 144,
 92,
 102,
 107,
 72,
 73,
 142,
 163,
 133,
 148,
 75,
 121,
 97,
 29,
 110,
 64,
 49,
 90,
 123,
 

In [ ]:
df_Bike_Station_month_usage_status = pd.DataFrame(Bike_Station_month_usage_status, columns=['이용 횟수(월)'])
df_Bike_Station_month_usage_status

,이용 횟수(월)
0,80
1,33
2,152
3,27
4,62
...,...
5611,133
5612,54
5613,157
5614,19


In [ ]:
df_total_Station_status = pd.DataFrame(total_Station_status, columns=['이용 횟수(전체)'])
df_total_Station_status

,이용 횟수(전체)
0,8366
1,4535
2,13402
3,2806
4,4444
...,...
151,7532
152,4190
153,473
154,952


In [ ]:
df_month_total_Station_status = pd.concat([df_all, df_Bike_Station_month_usage_status],axis=1)
df_month_total_Station_status # 월별 대여 이용 횟수

NameError: name 'df_all' is not defined

In [ ]:
df_all_total_Station_status = pd.concat([dfdf_station_num, df_total_Station_status],axis=1)
df_all_total_Station_status # 총 대여 이용 횟수

,이용 횟수(전체),이용 횟수(전체)
0,101,8366
1,103,4535
2,104,13402
3,105,2806
4,106,4444
...,...,...
151,349,7532
152,350,4190
153,351,473
154,992,952


In [ ]:
df_month_total_Station_status.to_csv('월별 대여 스테이션 이용 현황(완성).csv') # 저장
df_all_total_Station_status.to_csv('총 대여 스테이션 이용 현황(완성).csv')

## 반납 스테이션 현황

In [ ]:
df_all_re = pd.read_csv('월별 반납 스테이션 이용 현황(빈 리스트).csv')

In [ ]:
df_all_re

,Unnamed: 0,반납 시간(M),반납 스테이션 번호
0,0,2017-01,0
1,1,2017-01,101
2,2,2017-01,103
3,3,2017-01,104
4,4,2017-01,105
...,...,...,...
5683,153,2019-12,350
5684,154,2019-12,351
5685,155,2019-12,992
5686,156,2019-12,998


In [ ]:
return_month_hour = df['대여 시간(M)'].unique().tolist()
len(return_month_hour)

36

In [ ]:
re_station_num = df['반납 스테이션 번호'].unique().tolist()
re_station_num.sort() 
re_station_num

['0',
 '101',
 '103',
 '104',
 '105',
 '106',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '118',
 '119',
 '121',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '133',
 '137',
 '138',
 '139',
 '140',
 '141',
 '142',
 '143',
 '144',
 '146',
 '148',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '170',
 '171',
 '172',
 '173',
 '176',
 '177',
 '178',
 '201',
 '202',
 '203',
 '204',
 '205',
 '206',
 '207',
 '210',
 '211',
 '212',
 '213',
 '214',
 '215',
 '216',
 '217',
 '218',
 '219',
 '220',
 '221',
 '222',
 '223',
 '224',
 '225',
 '226',
 '227',
 '228',
 '229',
 '230',
 '231',
 '232',
 '233',
 '234',
 '235',
 '236',
 '237',
 '238',
 '239',
 '240',
 '245',
 '246',
 '247',
 '248',
 '249',
 '250',
 '251',
 '252',
 '253',
 '254',
 '255',
 '256',
 '257',
 '258',
 '259',
 '260',
 '261',
 '262',
 '263',
 '264',
 '265',
 '301',
 '302',
 '303',
 '304',
 '305',
 '306',
 '307',
 '308',
 '309',
 '310',
 '311',
 '312',
 '313',
 '314',
 '

In [ ]:
Bike_return_Station_month_usage_status = [] # 월별 반납 스테이션 이용 현황
for i in return_month_hour:
    for j in re_station_num:
        Bike_return_Station_month_usage_status.append(((df['대여 시간(M)']==i)&(df['반납 스테이션 번호']==j)).sum())
# 2017년도 2018년도 이런식으로 이용 현황으로 dataframe 만들어 줄 것

In [ ]:
len(Bike_return_Station_month_usage_status)

5688

In [ ]:
df['반납 스테이션 번호']=df[['반납 스테이션 번호']].astype('str')

In [ ]:
df['반납 스테이션 번호'].apply(lambda x: x.count('213')).sum() # 총 이용회수를 구할 수 있다.

25978

In [ ]:
((df['반납 스테이션 번호']=='110')&(df['대여 시간(M)']=='2017-01')).sum() # 월별 이용 회수 구하기

123

In [ ]:
# 총 대여 스테이션 이용 현황
total_re_Station_status = []
for i in re_station_num:
    total_re_Station_status.append(df['반납 스테이션 번호'].apply(lambda x: x.count(i)).sum())

In [ ]:
# 데이터 프레임 만들기
df_Bike_return_Station_month_usage_status = pd.DataFrame(Bike_return_Station_month_usage_status, columns=['이용 횟수(월)'])
df_total_re_Station_status = pd.DataFrame(total_re_Station_status, columns=['총 이용 횟수(월)'])

In [ ]:
df_Bike_return_Station_month_usage_status 

,이용 횟수(월)
0,3826
1,109
2,44
3,89
4,17
...,...
5683,37
5684,149
5685,19
5686,1


In [ ]:
return_station_num = df['반납 스테이션 번호'].unique().tolist()
return_station_num.sort() 
len(return_station_num)
df_re_station_num = pd.DataFrame(return_station_num, columns=['이용 횟수(전체)'])
df_re_station_num

,이용 횟수(전체)
0,0
1,101
2,103
3,104
4,105
...,...
153,350
154,351
155,992
156,998


In [ ]:
df_total_re_Station_status

,총 이용 횟수(월)
0,1174956
1,9207
2,5121
3,9330
4,2696
...,...
153,3662
154,406
155,795
156,21


In [ ]:
df_month_return_total_Station_status = pd.concat([df_all_re, df_Bike_return_Station_month_usage_status],axis=1)
df_month_return_total_Station_status[:159] # 월별 반납 스테이션 이용 횟수

,Unnamed: 0,반납 시간(M),반납 스테이션 번호,이용 횟수(월)
0,0,2017-01,0,3826
1,1,2017-01,101,109
2,2,2017-01,103,44
3,3,2017-01,104,89
4,4,2017-01,105,17
...,...,...,...,...
154,154,2017-01,351,0
155,155,2017-01,992,8
156,156,2017-01,998,0
157,157,2017-01,999,0


In [ ]:
df_all_rental_total_Station_status = pd.concat([df_re_station_num, df_total_re_Station_status],axis=1)
df_all_rental_total_Station_status # 총 반납 스테이션 이용 횟수

,이용 횟수(전체),총 이용 횟수(월)
0,0,1174956
1,101,9207
2,103,5121
3,104,9330
4,105,2696
...,...,...
153,350,3662
154,351,406
155,992,795
156,998,21


In [ ]:
df_month_return_total_Station_status.to_csv('월별 반납 스테이션 이용 현황(완성).csv') # 저장
df_all_rental_total_Station_status.to_csv('총 반납 스테이션 이용 현황(완성).csv')